In [1]:
%reload_ext autoreload
%autoreload 2
import datetime
import pathlib
import numpy
import os
import sklearn.preprocessing
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from classifier import classification_data

In [2]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "../features",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1
}
path = os.path.join(cfg["path_base"], "iwssip", "GRAYSCALE", "256")
file_input = os.path.join(cfg["path_base"], "handcraft", "MATLAB", "GRAYSCALE", "256", "surf64.txt")

In [ ]:
list_extractor = {
    "lbp": [-1],
    "surf64": [128, 256, -1],
    "surf128": [128, 256, -1]
}

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")


In [1]:
for p in pathlib.Path(path).rglob("*.txt"):
    data = numpy.loadtxt(file_input)
    dataset_name = str(p.stem)
    n_samples, n_features = data.shape
    n_features = n_features - 1
    x, y = data[0:, 0:n_features], data[:, n_features]
    x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

    get_dim = list(filter(lambda x: str(x) in str(p.stem), ["256", "400", "512"]))

    path = os.path.join(cfg["path_out"], current_datetime, dataset_name)
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

    print(x.shape, y.shape)

    for pca in list_extractor[str(p.stem)]:
        x = x_normalized
        if pca > 0:
            x = sklearn.decomposition.PCA(n_components=pca, random_state=cfg["random_state"]).fit_transform(x)

        if pca == -1:
            pca = n_features

        classification_data(cfg, dataset_name, get_dim, file_input, list(kf.split(x)), pca, n_features, n_samples, path, x, x, y)

NameError: name 'pathlib' is not defined